# A training institute which conducts training for analytics/ data science wants to expand their business to manpower recruitment (data science only) as well. 
 
Company gets large number of signups for their trainings. Now, company wants to connect these enrollees with their clients who are looking to hire employees working in the same domain. Before that, it is important to know which of these candidates are really looking for a new employment. They have student information related to demographics, education, experience and features related to training as well.
 
To understand the factors that lead a person to look for a job change, the agency wants you to design a model that uses the current credentials/demographics/experience to predict the probability of an enrollee to look for a new job.


# Data Dictionary Variable

Description

enrollee_id

Unique ID for enrollee

city

City code

city_development_index

Developement index of the city (scaled)

gender

Gender

relevent_experience

Relevent experience

enrolled_university

Type of University course enrolled if any

education_level

Education level

major_discipline

Major discipline

experience

Total experience in years

company_size

No of employees in current employer's company

company_type

Type of current employer

last_new_job

Difference in years between previous job and current job

training_hours

training hours completed

target

0 – Not looking for job change, 1 – Looking for a job change

 

Note that the dataset has a number of missing values and must be treated accordingly before fitting any model. You can refer to this article for a primer on how to deal with missing values.

sample_submission.csv


Column Name	Description
enrollee_id
Unique ID for enrollee

target

probability of an enrollee looking for a job change

# Evaluation Metric
The evaluation metric for this competition is AUC-ROC score.

In [1]:
# import standard libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score as rac
from sklearn.metrics import accuracy_score as ac
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# Read the training file and explore first few records
train = pd.read_csv("train_jqd04QH.csv")
train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [3]:
# read test file
test = pd.read_csv("test_KaymcHn.csv")
test.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,16548,city_33,0.448,NaN,No relevent experience,Full time course,Graduate,STEM,<1,1000-4999,Public Sector,NaN,15
1,12036,city_28,0.939,Male,No relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1,94
2,11061,city_103,0.920,Male,No relevent experience,Full time course,Graduate,STEM,3,NaN,NaN,1,17
3,5032,city_104,0.924,Male,No relevent experience,no_enrollment,Phd,STEM,>20,50-99,Pvt Ltd,2,76
4,17599,city_77,0.830,Male,Has relevent experience,no_enrollment,Graduate,STEM,6,<10,Pvt Ltd,2,65


In [4]:
train.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                      int64
dtype: object

In [5]:
train.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
target                       0
dtype: int64

In [6]:
train.shape

(18359, 14)

In [7]:
#explore categorical values , ones having missing values 
train.city.nunique()

123

In [8]:
train.gender.value_counts()

Male      12884
Female     1188
Other       189
Name: gender, dtype: int64

In [9]:
train.enrolled_university.value_counts()

no_enrollment       13659
Full time course     3187
Part time course     1171
Name: enrolled_university, dtype: int64

In [10]:
train.education_level.value_counts()

Graduate          10769
Masters            4319
High School        2032
Phd                 459
Primary School      323
Name: education_level, dtype: int64

In [11]:
train.major_discipline.value_counts()

STEM               13738
Humanities           688
Other                343
Business Degree      307
Arts                 239
No Major             206
Name: major_discipline, dtype: int64

In [12]:
train.experience.value_counts()

>20    3437
5      1309
4      1250
3      1159
6      1125
2       992
9       979
10      967
7       950
8       755
15      695
11      667
14      602
16      549
12      497
1       452
<1      416
13      412
17      347
19      308
18      286
20      146
Name: experience, dtype: int64

In [13]:
train.company_size.value_counts()

50-99        3120
100-500      2698
10000+       2044
10/49        1466
1000-4999    1399
<10          1360
500-999       902
5000-9999     591
Name: company_size, dtype: int64

In [14]:
train.company_type.value_counts()

Pvt Ltd                10051
Funded Startup          1038
Public Sector            996
Early Stage Startup      582
NGO                      534
Other                    119
Name: company_type, dtype: int64

In [15]:
train.last_new_job.value_counts()

1        7567
>4       3339
2        2835
never    2186
4        1038
3        1027
Name: last_new_job, dtype: int64

In [16]:
train.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target'],
      dtype='object')

In [17]:
test.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
dtype: object

In [18]:
test.shape

(15021, 13)

In [19]:
test.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    3388
relevent_experience          0
enrolled_university        279
education_level            395
major_discipline          2393
experience                  44
company_size              4051
company_type              4330
last_new_job               304
training_hours               0
dtype: int64

In [20]:
#missing value imputation in the training data
# impute last_new_job with values never
# gender value with female
# enrolled_university to no_enrollment
# major_discipline to STEM
# company type pvt. Lts
# company size to 500 - 999
# education_level to Masters

In [21]:
train['last_new_job']        = train['last_new_job'].fillna('never')
train['gender']              = train['gender'].fillna('Female')
train['enrolled_university'] = train['enrolled_university'].fillna('no_enrollment')
train['major_discipline']    = train['major_discipline'].fillna('STEM')
train['company_type']        = train['company_type'].fillna('Pvt Ltd')
train['company_size']        = train['company_size'].fillna('500-999')
train['education_level']     = train['education_level'].fillna('Masters')
train['experience']          = train['experience'].fillna('>20')

In [22]:
test['last_new_job']        = test['last_new_job'].fillna('never')
test['gender']              = test['gender'].fillna('Female')
test['enrolled_university'] = test['enrolled_university'].fillna('no_enrollment')
test['major_discipline']    = test['major_discipline'].fillna('STEM')
test['company_type']        = test['company_type'].fillna('Pvt Ltd')
test['company_size']        = test['company_size'].fillna('500-999')
test['education_level']     = test['education_level'].fillna('Masters')
test['experience']          = test['experience'].fillna('>20')

In [23]:
train.isnull().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

In [24]:
test.isnull().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
dtype: int64

In [25]:
train.target.value_counts()

0    15934
1     2425
Name: target, dtype: int64

In [26]:
# upsample the minority class to balance the data
from sklearn.utils import resample
df0 = train[train.target ==0]
df1 = train[train.target ==1]
df1r = resample(df1,replace=True,n_samples = 15934,random_state = 11)
dfus = pd.concat([df0,df1r])
dfus.target.value_counts()

1    15934
0    15934
Name: target, dtype: int64

In [27]:
traindf = pd.get_dummies(dfus)

In [28]:
traindf.head()

,enrollee_id,city_development_index,training_hours,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,...,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,last_new_job_1,last_new_job_2,last_new_job_3,last_new_job_4,last_new_job_>4,last_new_job_never
0,23798,0.689,106,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,29166,0.923,69,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,46,0.910,4,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,18527,0.666,26,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
5,13342,0.624,34,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [29]:
testdf = pd.get_dummies(test)

In [30]:
testdf.head()

,enrollee_id,city_development_index,training_hours,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,city_city_104,...,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,last_new_job_1,last_new_job_2,last_new_job_3,last_new_job_4,last_new_job_>4,last_new_job_never
0,16548,0.448,15,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,12036,0.939,94,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2,11061,0.920,17,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,5032,0.924,76,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
4,17599,0.830,65,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [ ]:
# label encode categorical variables

le =  preprocessing.LabelEncoder()

train['last_new_job']        = le.fit_transform(train['last_new_job'])
train['gender']              = le.fit_transform(train['gender'])
train['enrolled_university'] = le.fit_transform(train['enrolled_university'])
train['major_discipline']    = le.fit_transform(train['major_discipline'])
train['company_type']        = le.fit_transform(train['company_type'])
train['company_size']        = le.fit_transform(train['company_size'])
train['education_level']     = le.fit_transform(train['education_level'])
train['experience']          = le.fit_transform(train['experience'])
train['city']                = le.fit_transform(train['city'])
train['relevent_experience'] = le.fit_transform(train['relevent_experience'])

In [ ]:
test['last_new_job']        = le.fit_transform(test['last_new_job'])
test['gender']              = le.fit_transform(test['gender'])
test['enrolled_university'] = le.fit_transform(test['enrolled_university'])
test['major_discipline']    = le.fit_transform(test['major_discipline'])
test['company_type']        = le.fit_transform(test['company_type'])
test['company_size']        = le.fit_transform(test['company_size'])
test['education_level']     = le.fit_transform(test['education_level'])
test['experience']          = le.fit_transform(test['experience'])
test['city']                = le.fit_transform(test['city'])
test['relevent_experience'] = le.fit_transform(test['relevent_experience'])

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['target'].value_counts('normalize')

In [ ]:
fig = plt.figure(figsize = (12,6))
sns.barplot(train['city'],train['target'].groupby(train['city']).sum())

In [ ]:
trainX = train[['city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours']].values

trainy = train['target'].values

In [31]:
trainX = traindf.drop(columns = ['enrollee_id','target'])

In [32]:
trainX.shape

(31868, 186)

In [33]:
testX = testdf.drop(columns = ['enrollee_id'])
testX.shape

(15021, 186)

In [34]:
X = trainX.values
X

array([[  0.689, 106.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.923,  69.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.91 ,   4.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       ...,
       [  0.91 ,  64.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.926,  45.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.698, 135.   ,   0.   , ...,   0.   ,   0.   ,   0.   ]])

In [35]:
Xt = testX.values
Xt

array([[  0.448,  15.   ,   0.   , ...,   0.   ,   0.   ,   1.   ],
       [  0.939,  94.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.92 ,  17.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       ...,
       [  0.91 ,  13.   ,   0.   , ...,   1.   ,   0.   ,   0.   ],
       [  0.666,  38.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.624, 100.   ,   0.   , ...,   0.   ,   0.   ,   1.   ]])

In [36]:
testX.shape

(15021, 186)

In [37]:
y = traindf['target'].values
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [ ]:
test.columns

In [ ]:
testX = test[['city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours']].values

In [ ]:
trainX

In [ ]:
testX

In [ ]:
trainy

In [ ]:
sc = preprocessing.MinMaxScaler()
X_train_scale = sc.fit_transform(trainX)

In [38]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size =0.30,shuffle =True,random_state = 42,stratify=y)
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((22307, 186), (9561, 186), (22307,), (9561,))

In [ ]:
np.where(y_train ==0) 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
T = [0.95,0.60,0.80,.85,.90,.86,.87,.88,.89]
lrpT =[]
#C_param_range = [0.001,0.01,0.1,1,10,100,1000]
#for c in C_param_range :
lrcl = LogisticRegression()
lrcl.fit(X_train,y_train)
lrp = lrcl.predict_proba(X_val)
print(lrp[:,0])    
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, lrp[:,0])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

for t in T :
    for p in lrp[:,0] :
        if p <= t :
            lrpT.append(1)
        else:
            lrpT.append(0)
    lrpt = np.array(lrpT)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, lrpT)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    print(t,roc_auc)
    lrpT =[]

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
rfcl = RandomForestClassifier(n_estimators = 5000,max_depth = 5,min_samples_split = 500,
                              max_features = 10)
rfcl.fit(X_train,y_train)
rfp = rfcl.predict_proba(X_val)
print(rfp[:,1])    
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, rfp[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

[0.56717863 0.45157015 0.59152898 ... 0.52936086 0.50502294 0.49475774]
0.6672366821597694


In [54]:
import xgboost
from xgboost import XGBClassifier
xgbcl = XGBClassifier(objective = 'binary:logistic',eta=0.2,max_depth =5,n_estimators = 100)
xgbcl.fit(X_train,y_train)
xgp = xgbcl.predict_proba(X_val)
print(xgp[:,1])    
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, xgp[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

[0.5972552  0.3728011  0.6406143  ... 0.60835063 0.54422003 0.26113388]
0.7504102054943776


In [57]:
xgp = xgbcl.predict(Xt)
xgp

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [40]:
from catboost import CatBoostClassifier
catbcl = CatBoostClassifier()
catbcl.fit(X_train,y_train)
catbp = catbcl.predict_proba(X_val)
print(catbp[:,1])    
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, catbp[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

Learning rate set to 0.03879
0:	learn: 0.6900709	total: 171ms	remaining: 2m 51s
1:	learn: 0.6872963	total: 186ms	remaining: 1m 32s
2:	learn: 0.6847840	total: 203ms	remaining: 1m 7s
3:	learn: 0.6822981	total: 216ms	remaining: 53.8s
4:	learn: 0.6801334	total: 230ms	remaining: 45.7s
5:	learn: 0.6779696	total: 245ms	remaining: 40.7s
6:	learn: 0.6759361	total: 258ms	remaining: 36.5s
7:	learn: 0.6741564	total: 271ms	remaining: 33.6s
8:	learn: 0.6725019	total: 283ms	remaining: 31.1s
9:	learn: 0.6708243	total: 295ms	remaining: 29.2s
10:	learn: 0.6692091	total: 309ms	remaining: 27.8s
11:	learn: 0.6675904	total: 321ms	remaining: 26.4s
12:	learn: 0.6664054	total: 333ms	remaining: 25.3s
13:	learn: 0.6649711	total: 345ms	remaining: 24.3s
14:	learn: 0.6637327	total: 357ms	remaining: 23.4s
15:	learn: 0.6625154	total: 371ms	remaining: 22.8s
16:	learn: 0.6611506	total: 383ms	remaining: 22.1s
17:	learn: 0.6599309	total: 395ms	remaining: 21.5s
18:	learn: 0.6589778	total: 407ms	remaining: 21s
19:	learn: 0

160:	learn: 0.6090733	total: 2.34s	remaining: 12.2s
161:	learn: 0.6087412	total: 2.36s	remaining: 12.2s
162:	learn: 0.6084064	total: 2.37s	remaining: 12.2s
163:	learn: 0.6079904	total: 2.38s	remaining: 12.1s
164:	learn: 0.6078046	total: 2.4s	remaining: 12.1s
165:	learn: 0.6075895	total: 2.41s	remaining: 12.1s
166:	learn: 0.6074171	total: 2.42s	remaining: 12.1s
167:	learn: 0.6071929	total: 2.44s	remaining: 12.1s
168:	learn: 0.6068633	total: 2.45s	remaining: 12s
169:	learn: 0.6066389	total: 2.46s	remaining: 12s
170:	learn: 0.6064689	total: 2.47s	remaining: 12s
171:	learn: 0.6060846	total: 2.49s	remaining: 12s
172:	learn: 0.6058631	total: 2.51s	remaining: 12s
173:	learn: 0.6055987	total: 2.53s	remaining: 12s
174:	learn: 0.6053663	total: 2.55s	remaining: 12s
175:	learn: 0.6050424	total: 2.56s	remaining: 12s
176:	learn: 0.6049290	total: 2.57s	remaining: 12s
177:	learn: 0.6046391	total: 2.59s	remaining: 11.9s
178:	learn: 0.6043279	total: 2.6s	remaining: 11.9s
179:	learn: 0.6041140	total: 2.6

325:	learn: 0.5712592	total: 5.11s	remaining: 10.6s
326:	learn: 0.5710289	total: 5.13s	remaining: 10.5s
327:	learn: 0.5708494	total: 5.15s	remaining: 10.6s
328:	learn: 0.5706420	total: 5.17s	remaining: 10.5s
329:	learn: 0.5703616	total: 5.18s	remaining: 10.5s
330:	learn: 0.5700710	total: 5.2s	remaining: 10.5s
331:	learn: 0.5697863	total: 5.23s	remaining: 10.5s
332:	learn: 0.5695889	total: 5.24s	remaining: 10.5s
333:	learn: 0.5693838	total: 5.25s	remaining: 10.5s
334:	learn: 0.5693272	total: 5.26s	remaining: 10.4s
335:	learn: 0.5691828	total: 5.28s	remaining: 10.4s
336:	learn: 0.5689035	total: 5.29s	remaining: 10.4s
337:	learn: 0.5686335	total: 5.3s	remaining: 10.4s
338:	learn: 0.5684432	total: 5.31s	remaining: 10.4s
339:	learn: 0.5681965	total: 5.33s	remaining: 10.3s
340:	learn: 0.5679964	total: 5.34s	remaining: 10.3s
341:	learn: 0.5678544	total: 5.35s	remaining: 10.3s
342:	learn: 0.5675897	total: 5.36s	remaining: 10.3s
343:	learn: 0.5673983	total: 5.37s	remaining: 10.2s
344:	learn: 0.

489:	learn: 0.5391179	total: 7.68s	remaining: 7.99s
490:	learn: 0.5389164	total: 7.7s	remaining: 7.98s
491:	learn: 0.5387561	total: 7.72s	remaining: 7.97s
492:	learn: 0.5385554	total: 7.75s	remaining: 7.96s
493:	learn: 0.5383770	total: 7.76s	remaining: 7.95s
494:	learn: 0.5381283	total: 7.77s	remaining: 7.93s
495:	learn: 0.5379342	total: 7.78s	remaining: 7.91s
496:	learn: 0.5378155	total: 7.79s	remaining: 7.89s
497:	learn: 0.5375166	total: 7.81s	remaining: 7.87s
498:	learn: 0.5372837	total: 7.82s	remaining: 7.85s
499:	learn: 0.5372574	total: 7.83s	remaining: 7.83s
500:	learn: 0.5370542	total: 7.85s	remaining: 7.82s
501:	learn: 0.5368726	total: 7.88s	remaining: 7.81s
502:	learn: 0.5367381	total: 7.89s	remaining: 7.79s
503:	learn: 0.5366308	total: 7.9s	remaining: 7.78s
504:	learn: 0.5364111	total: 7.95s	remaining: 7.79s
505:	learn: 0.5362023	total: 7.96s	remaining: 7.78s
506:	learn: 0.5359938	total: 7.98s	remaining: 7.76s
507:	learn: 0.5358300	total: 7.99s	remaining: 7.74s
508:	learn: 0.

659:	learn: 0.5118035	total: 10s	remaining: 5.17s
660:	learn: 0.5116320	total: 10s	remaining: 5.15s
661:	learn: 0.5115279	total: 10.1s	remaining: 5.13s
662:	learn: 0.5114099	total: 10.1s	remaining: 5.12s
663:	learn: 0.5111998	total: 10.1s	remaining: 5.1s
664:	learn: 0.5110605	total: 10.1s	remaining: 5.08s
665:	learn: 0.5110245	total: 10.1s	remaining: 5.07s
666:	learn: 0.5108950	total: 10.1s	remaining: 5.05s
667:	learn: 0.5107678	total: 10.1s	remaining: 5.04s
668:	learn: 0.5105905	total: 10.1s	remaining: 5.02s
669:	learn: 0.5105090	total: 10.2s	remaining: 5s
670:	learn: 0.5104822	total: 10.2s	remaining: 4.99s
671:	learn: 0.5102372	total: 10.2s	remaining: 4.97s
672:	learn: 0.5100631	total: 10.2s	remaining: 4.96s
673:	learn: 0.5099224	total: 10.2s	remaining: 4.94s
674:	learn: 0.5097968	total: 10.2s	remaining: 4.93s
675:	learn: 0.5097788	total: 10.2s	remaining: 4.91s
676:	learn: 0.5097032	total: 10.3s	remaining: 4.89s
677:	learn: 0.5096092	total: 10.3s	remaining: 4.88s
678:	learn: 0.509525

820:	learn: 0.4883458	total: 12.4s	remaining: 2.7s
821:	learn: 0.4882119	total: 12.4s	remaining: 2.68s
822:	learn: 0.4880934	total: 12.4s	remaining: 2.67s
823:	learn: 0.4879770	total: 12.4s	remaining: 2.65s
824:	learn: 0.4878184	total: 12.4s	remaining: 2.63s
825:	learn: 0.4876665	total: 12.4s	remaining: 2.62s
826:	learn: 0.4875336	total: 12.4s	remaining: 2.6s
827:	learn: 0.4874842	total: 12.5s	remaining: 2.59s
828:	learn: 0.4874168	total: 12.5s	remaining: 2.58s
829:	learn: 0.4872470	total: 12.5s	remaining: 2.56s
830:	learn: 0.4871914	total: 12.5s	remaining: 2.55s
831:	learn: 0.4870041	total: 12.5s	remaining: 2.53s
832:	learn: 0.4868249	total: 12.6s	remaining: 2.52s
833:	learn: 0.4866339	total: 12.6s	remaining: 2.5s
834:	learn: 0.4864621	total: 12.6s	remaining: 2.49s
835:	learn: 0.4863491	total: 12.6s	remaining: 2.47s
836:	learn: 0.4861834	total: 12.6s	remaining: 2.46s
837:	learn: 0.4860151	total: 12.6s	remaining: 2.44s
838:	learn: 0.4858693	total: 12.6s	remaining: 2.42s
839:	learn: 0.4

983:	learn: 0.4671533	total: 14.7s	remaining: 239ms
984:	learn: 0.4670149	total: 14.7s	remaining: 224ms
985:	learn: 0.4668322	total: 14.8s	remaining: 209ms
986:	learn: 0.4667358	total: 14.8s	remaining: 195ms
987:	learn: 0.4666132	total: 14.8s	remaining: 180ms
988:	learn: 0.4664287	total: 14.8s	remaining: 165ms
989:	learn: 0.4663685	total: 14.9s	remaining: 150ms
990:	learn: 0.4662019	total: 14.9s	remaining: 135ms
991:	learn: 0.4661614	total: 14.9s	remaining: 120ms
992:	learn: 0.4660271	total: 14.9s	remaining: 105ms
993:	learn: 0.4657837	total: 15s	remaining: 90.4ms
994:	learn: 0.4656733	total: 15s	remaining: 75.3ms
995:	learn: 0.4655053	total: 15s	remaining: 60.3ms
996:	learn: 0.4653475	total: 15s	remaining: 45.3ms
997:	learn: 0.4653291	total: 15.1s	remaining: 30.2ms
998:	learn: 0.4652823	total: 15.1s	remaining: 15.1ms
999:	learn: 0.4650992	total: 15.1s	remaining: 0us
[0.52485559 0.340088   0.728318   ... 0.61573655 0.64483803 0.26886601]
0.8464498594943899


In [41]:
catp = catbcl.predict(Xt)
catp

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [42]:
from sklearn.model_selection import cross_val_score,cross_val_predict

crossval_scores = cross_val_score(catbcl, X, y, scoring='roc_auc', cv=10)
print(crossval_scores)
cv_predict_val = cross_val_predict(catbcl,X,y,cv =10)
catbcl.fit(X,y)
catp = catbcl.predict(Xt)

Learning rate set to 0.043184
0:	learn: 0.6897817	total: 15.2ms	remaining: 15.2s
1:	learn: 0.6871121	total: 26.8ms	remaining: 13.4s
2:	learn: 0.6843140	total: 40.6ms	remaining: 13.5s
3:	learn: 0.6817116	total: 52.7ms	remaining: 13.1s
4:	learn: 0.6793761	total: 65.7ms	remaining: 13.1s
5:	learn: 0.6772670	total: 78.8ms	remaining: 13.1s
6:	learn: 0.6752605	total: 92ms	remaining: 13.1s
7:	learn: 0.6730769	total: 105ms	remaining: 13.1s
8:	learn: 0.6711001	total: 118ms	remaining: 13s
9:	learn: 0.6692102	total: 129ms	remaining: 12.8s
10:	learn: 0.6674701	total: 142ms	remaining: 12.8s
11:	learn: 0.6661114	total: 155ms	remaining: 12.8s
12:	learn: 0.6647247	total: 177ms	remaining: 13.4s
13:	learn: 0.6632770	total: 194ms	remaining: 13.7s
14:	learn: 0.6622260	total: 214ms	remaining: 14.1s
15:	learn: 0.6610440	total: 228ms	remaining: 14s
16:	learn: 0.6597989	total: 241ms	remaining: 13.9s
17:	learn: 0.6589059	total: 254ms	remaining: 13.9s
18:	learn: 0.6578115	total: 267ms	remaining: 13.8s
19:	learn:

166:	learn: 0.6033992	total: 2.52s	remaining: 12.6s
167:	learn: 0.6031723	total: 2.53s	remaining: 12.5s
168:	learn: 0.6030279	total: 2.55s	remaining: 12.6s
169:	learn: 0.6028251	total: 2.58s	remaining: 12.6s
170:	learn: 0.6024681	total: 2.6s	remaining: 12.6s
171:	learn: 0.6022468	total: 2.61s	remaining: 12.6s
172:	learn: 0.6019240	total: 2.63s	remaining: 12.6s
173:	learn: 0.6017051	total: 2.65s	remaining: 12.6s
174:	learn: 0.6014826	total: 2.66s	remaining: 12.5s
175:	learn: 0.6011687	total: 2.67s	remaining: 12.5s
176:	learn: 0.6007481	total: 2.69s	remaining: 12.5s
177:	learn: 0.6003685	total: 2.7s	remaining: 12.5s
178:	learn: 0.6002470	total: 2.74s	remaining: 12.6s
179:	learn: 0.6000119	total: 2.75s	remaining: 12.5s
180:	learn: 0.5996730	total: 2.77s	remaining: 12.5s
181:	learn: 0.5994389	total: 2.79s	remaining: 12.6s
182:	learn: 0.5992517	total: 2.82s	remaining: 12.6s
183:	learn: 0.5990444	total: 2.84s	remaining: 12.6s
184:	learn: 0.5988447	total: 2.87s	remaining: 12.7s
185:	learn: 0.

326:	learn: 0.5652054	total: 5.43s	remaining: 11.2s
327:	learn: 0.5649910	total: 5.45s	remaining: 11.2s
328:	learn: 0.5648003	total: 5.46s	remaining: 11.1s
329:	learn: 0.5644676	total: 5.48s	remaining: 11.1s
330:	learn: 0.5644139	total: 5.5s	remaining: 11.1s
331:	learn: 0.5641661	total: 5.51s	remaining: 11.1s
332:	learn: 0.5640102	total: 5.53s	remaining: 11.1s
333:	learn: 0.5638630	total: 5.55s	remaining: 11.1s
334:	learn: 0.5637759	total: 5.56s	remaining: 11s
335:	learn: 0.5635646	total: 5.58s	remaining: 11s
336:	learn: 0.5633637	total: 5.59s	remaining: 11s
337:	learn: 0.5631577	total: 5.61s	remaining: 11s
338:	learn: 0.5629879	total: 5.62s	remaining: 11s
339:	learn: 0.5627324	total: 5.66s	remaining: 11s
340:	learn: 0.5624308	total: 5.68s	remaining: 11s
341:	learn: 0.5623924	total: 5.71s	remaining: 11s
342:	learn: 0.5621384	total: 5.74s	remaining: 11s
343:	learn: 0.5619569	total: 5.76s	remaining: 11s
344:	learn: 0.5616417	total: 5.78s	remaining: 11s
345:	learn: 0.5613568	total: 5.79s	

491:	learn: 0.5309449	total: 8.33s	remaining: 8.61s
492:	learn: 0.5306805	total: 8.36s	remaining: 8.59s
493:	learn: 0.5304266	total: 8.37s	remaining: 8.57s
494:	learn: 0.5302553	total: 8.39s	remaining: 8.56s
495:	learn: 0.5301202	total: 8.41s	remaining: 8.54s
496:	learn: 0.5299381	total: 8.42s	remaining: 8.53s
497:	learn: 0.5297588	total: 8.44s	remaining: 8.5s
498:	learn: 0.5295155	total: 8.45s	remaining: 8.48s
499:	learn: 0.5292303	total: 8.46s	remaining: 8.46s
500:	learn: 0.5290582	total: 8.47s	remaining: 8.44s
501:	learn: 0.5288101	total: 8.49s	remaining: 8.42s
502:	learn: 0.5285794	total: 8.51s	remaining: 8.4s
503:	learn: 0.5283581	total: 8.53s	remaining: 8.39s
504:	learn: 0.5281760	total: 8.55s	remaining: 8.38s
505:	learn: 0.5278678	total: 8.56s	remaining: 8.36s
506:	learn: 0.5278479	total: 8.58s	remaining: 8.34s
507:	learn: 0.5276392	total: 8.59s	remaining: 8.32s
508:	learn: 0.5274328	total: 8.6s	remaining: 8.3s
509:	learn: 0.5272317	total: 8.62s	remaining: 8.28s
510:	learn: 0.52

652:	learn: 0.5017468	total: 11.7s	remaining: 6.19s
653:	learn: 0.5016066	total: 11.7s	remaining: 6.19s
654:	learn: 0.5015013	total: 11.7s	remaining: 6.17s
655:	learn: 0.5013242	total: 11.7s	remaining: 6.16s
656:	learn: 0.5011672	total: 11.8s	remaining: 6.15s
657:	learn: 0.5010390	total: 11.8s	remaining: 6.14s
658:	learn: 0.5009183	total: 11.8s	remaining: 6.13s
659:	learn: 0.5007964	total: 11.9s	remaining: 6.12s
660:	learn: 0.5005876	total: 11.9s	remaining: 6.1s
661:	learn: 0.5004075	total: 11.9s	remaining: 6.08s
662:	learn: 0.5002922	total: 11.9s	remaining: 6.06s
663:	learn: 0.5001959	total: 11.9s	remaining: 6.04s
664:	learn: 0.5000745	total: 12s	remaining: 6.02s
665:	learn: 0.4998231	total: 12s	remaining: 6s
666:	learn: 0.4996243	total: 12s	remaining: 5.99s
667:	learn: 0.4994592	total: 12s	remaining: 5.97s
668:	learn: 0.4993057	total: 12s	remaining: 5.95s
669:	learn: 0.4991239	total: 12s	remaining: 5.93s
670:	learn: 0.4990124	total: 12.1s	remaining: 5.91s
671:	learn: 0.4989136	total:

...

200:	learn: 0.5939998	total: 3.52s	remaining: 14s
201:	learn: 0.5936678	total: 3.54s	remaining: 14s
202:	learn: 0.5934514	total: 3.55s	remaining: 14s
203:	learn: 0.5932341	total: 3.57s	remaining: 13.9s
204:	learn: 0.5929566	total: 3.58s	remaining: 13.9s
205:	learn: 0.5927628	total: 3.6s	remaining: 13.9s
206:	learn: 0.5924946	total: 3.61s	remaining: 13.8s
207:	learn: 0.5922328	total: 3.63s	remaining: 13.8s
208:	learn: 0.5920729	total: 3.65s	remaining: 13.8s
209:	learn: 0.5917861	total: 3.67s	remaining: 13.8s
210:	learn: 0.5915625	total: 3.68s	remaining: 13.8s
211:	learn: 0.5913220	total: 3.7s	remaining: 13.8s
212:	learn: 0.5911170	total: 3.72s	remaining: 13.8s
213:	learn: 0.5905993	total: 3.74s	remaining: 13.7s
214:	learn: 0.5904154	total: 3.75s	remaining: 13.7s
215:	learn: 0.5902446	total: 3.77s	remaining: 13.7s
216:	learn: 0.5899732	total: 3.79s	remaining: 13.7s
217:	learn: 0.5896805	total: 3.8s	remaining: 13.6s
218:	learn: 0.5894885	total: 3.82s	remaining: 13.6s
219:	learn: 0.5892222

370:	learn: 0.5526432	total: 6.24s	remaining: 10.6s
371:	learn: 0.5523955	total: 6.25s	remaining: 10.6s
372:	learn: 0.5521986	total: 6.27s	remaining: 10.5s
373:	learn: 0.5518874	total: 6.28s	remaining: 10.5s
374:	learn: 0.5516683	total: 6.29s	remaining: 10.5s
375:	learn: 0.5514353	total: 6.32s	remaining: 10.5s
376:	learn: 0.5512383	total: 6.34s	remaining: 10.5s
377:	learn: 0.5510447	total: 6.35s	remaining: 10.5s
378:	learn: 0.5508697	total: 6.37s	remaining: 10.4s
379:	learn: 0.5506558	total: 6.38s	remaining: 10.4s
380:	learn: 0.5503016	total: 6.39s	remaining: 10.4s
381:	learn: 0.5501005	total: 6.41s	remaining: 10.4s
382:	learn: 0.5499417	total: 6.43s	remaining: 10.4s
383:	learn: 0.5497376	total: 6.44s	remaining: 10.3s
384:	learn: 0.5495465	total: 6.46s	remaining: 10.3s
385:	learn: 0.5493031	total: 6.47s	remaining: 10.3s
386:	learn: 0.5491416	total: 6.49s	remaining: 10.3s
387:	learn: 0.5490193	total: 6.51s	remaining: 10.3s
388:	learn: 0.5488330	total: 6.53s	remaining: 10.3s
389:	learn: 

531:	learn: 0.5214422	total: 8.79s	remaining: 7.73s
532:	learn: 0.5212432	total: 8.8s	remaining: 7.71s
533:	learn: 0.5210663	total: 8.82s	remaining: 7.7s
534:	learn: 0.5209136	total: 8.83s	remaining: 7.68s
535:	learn: 0.5207189	total: 8.85s	remaining: 7.66s
536:	learn: 0.5205472	total: 8.87s	remaining: 7.65s
537:	learn: 0.5203704	total: 8.89s	remaining: 7.63s
538:	learn: 0.5201491	total: 8.9s	remaining: 7.62s
539:	learn: 0.5198914	total: 8.92s	remaining: 7.6s
540:	learn: 0.5197604	total: 8.93s	remaining: 7.58s
541:	learn: 0.5196325	total: 8.95s	remaining: 7.56s
542:	learn: 0.5194897	total: 8.96s	remaining: 7.54s
543:	learn: 0.5192341	total: 8.98s	remaining: 7.53s
544:	learn: 0.5190909	total: 8.99s	remaining: 7.51s
545:	learn: 0.5188910	total: 9.01s	remaining: 7.49s
546:	learn: 0.5187056	total: 9.02s	remaining: 7.47s
547:	learn: 0.5185001	total: 9.05s	remaining: 7.46s
548:	learn: 0.5182423	total: 9.06s	remaining: 7.44s
549:	learn: 0.5179203	total: 9.07s	remaining: 7.42s
550:	learn: 0.51

692:	learn: 0.4936145	total: 11.3s	remaining: 5.01s
693:	learn: 0.4934430	total: 11.3s	remaining: 5s
694:	learn: 0.4932877	total: 11.4s	remaining: 4.98s
695:	learn: 0.4930524	total: 11.4s	remaining: 4.96s
696:	learn: 0.4928963	total: 11.4s	remaining: 4.95s
697:	learn: 0.4927062	total: 11.4s	remaining: 4.93s
698:	learn: 0.4925930	total: 11.4s	remaining: 4.91s
699:	learn: 0.4925199	total: 11.4s	remaining: 4.9s
700:	learn: 0.4924334	total: 11.4s	remaining: 4.88s
701:	learn: 0.4921818	total: 11.5s	remaining: 4.86s
702:	learn: 0.4920648	total: 11.5s	remaining: 4.85s
703:	learn: 0.4919043	total: 11.5s	remaining: 4.83s
704:	learn: 0.4917799	total: 11.5s	remaining: 4.82s
705:	learn: 0.4915810	total: 11.5s	remaining: 4.8s
706:	learn: 0.4914204	total: 11.5s	remaining: 4.78s
707:	learn: 0.4912365	total: 11.6s	remaining: 4.77s
708:	learn: 0.4910338	total: 11.6s	remaining: 4.75s
709:	learn: 0.4908321	total: 11.6s	remaining: 4.73s
710:	learn: 0.4907380	total: 11.6s	remaining: 4.72s
711:	learn: 0.490

854:	learn: 0.4702546	total: 13.9s	remaining: 2.35s
855:	learn: 0.4701312	total: 13.9s	remaining: 2.34s
856:	learn: 0.4699431	total: 13.9s	remaining: 2.32s
857:	learn: 0.4698417	total: 13.9s	remaining: 2.3s
858:	learn: 0.4696790	total: 13.9s	remaining: 2.29s
859:	learn: 0.4695836	total: 13.9s	remaining: 2.27s
860:	learn: 0.4694566	total: 14s	remaining: 2.25s
861:	learn: 0.4692871	total: 14s	remaining: 2.24s
862:	learn: 0.4692287	total: 14s	remaining: 2.22s
863:	learn: 0.4691676	total: 14s	remaining: 2.21s
864:	learn: 0.4690706	total: 14s	remaining: 2.19s
865:	learn: 0.4689355	total: 14s	remaining: 2.17s
866:	learn: 0.4687079	total: 14.1s	remaining: 2.16s
867:	learn: 0.4686093	total: 14.1s	remaining: 2.14s
868:	learn: 0.4684524	total: 14.1s	remaining: 2.12s
869:	learn: 0.4682907	total: 14.1s	remaining: 2.11s
870:	learn: 0.4682796	total: 14.1s	remaining: 2.09s
871:	learn: 0.4681940	total: 14.1s	remaining: 2.07s
872:	learn: 0.4680656	total: 14.1s	remaining: 2.06s
873:	learn: 0.4679582	tot

In [43]:
print(crossval_scores)

[0.87939481 0.87198977 0.85668929 0.86184547 0.88566563 0.88063716
 0.86988685 0.863529   0.86653434 0.86369549]


In [44]:
catp

array([1, 0, 1, ..., 0, 0, 1], dtype=int64)

In [ ]:
svcl = SVC(gamma ='auto',kernel = 'linear',class_weight = 'balanced',probability =True)
svcl.fit(X_train,y_train)
svp = svcl.predict_proba(X_val)
print(svp[:,1])    
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, svp[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

In [ ]:
from sklearn.naive_bayes import GaussianNB
nbcl = GaussianNB()
nbcl.fit(X_train,y_train)
nbp = nbcl.predict_proba(X_val)
print(nbp[:,1])    
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, nbp[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

In [ ]:
rfp = rfcl.predict(Xt)
rfp

In [ ]:
from sklearn.linear_model import LogisticRegression
lrcl = LogisticRegression()
lrcl.fit(X_train,y_train)
lrp = lrcl.predict_proba(X_val)
print(lrp[:,1])    
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, lrp[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

In [ ]:
lrp = lrcl.predict(Xt)
lrp

In [ ]:
np.where(rfp > 0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,cross_val_predict

#X = trainX
#y = trainy
crossval_scores = cross_val_score(LogisticRegression(), X, y, scoring='roc_auc', cv=10)
cv_predict_val = cross_val_predict(LogisticRegression(),X,y,cv =10)
lrm = LogisticRegression().fit(X,y)
lrp = lrm.predict(testX)

In [ ]:
np.mean(crossval_scores)

In [ ]:
cv_predict_val

In [ ]:
lrp

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

X = trainX
y = trainy
crossval_scores = cross_val_score(KNeighborsClassifier(n_neighbors=7), X, y, scoring='roc_auc', cv=10)
cv_predict_val = cross_val_predict(KNeighborsClassifier(n_neighbors=7),X,y,cv =10)
knnm = KNeighborsClassifier(n_neighbors=3).fit(X,y)
knnp = knnm.predict(testX)

In [ ]:
np.mean(crossval_scores)

In [ ]:
knnp

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X = trainX
y = trainy
crossval_scores = cross_val_score(RandomForestClassifier(n_estimators = 1000), X, y, scoring='roc_auc', cv=10)
cv_predict_val = cross_val_predict(RandomForestClassifier(),X,y,cv =10)
rfm = RandomForestClassifier().fit(X,y)
rfmp = rfm.predict(testX)

In [ ]:
np.mean(crossval_scores)

In [ ]:
rfmp

In [ ]:
cv = StratifiedKFold(n_splits =5)
classifier = SVC(kernel='linear', probability=True,
                     random_state=11)

X = trainX
y = trainy
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots()
for i, (train_X, train_y) in enumerate(cv.split(X, y)):
    classifier.fit(X[train_X], y[train_X])
    viz = plot_roc_curve(classifier, X[train_y], y[train_y],
                         name='ROC fold {}'.format(i),
                         alpha=0.3, lw=1, ax=ax)
    interp_tpr = interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic example")
ax.legend(loc="lower right")
plt.show()

In [ ]:
X_train

In [ ]:
svcls = SVC(gamma ='auto')
svcls.fit(X_train,y_train)
svp = svcls.predict(X_val)
rac(y_val,svp),ac(y_val,svp)

In [ ]:
lrp

In [ ]:
lprob = lrcls.predict_proba(X_val_scale)

In [ ]:
lprob

In [ ]:
Thres = 0.35
lrpp =[]
while Thres <= 0.50 :
    
    pp = lrcls.predict_proba(X_val)
    for p in pp[:,0] :
        if p >= Thres :
            lrpp.append(1)
        else:
            lrpp.append(0)
    lrpp = lrcls.predict(X_val)
    print(Thres,rac(y_val,lrpp),ac(y_val,lrpp))
    lrpp =[]
    Thres+= 0.01

In [45]:
sam = pd.read_csv("sample_submission_sxfcbdx.csv")
sam.head()

,enrollee_id,target
0,16548,0
1,12036,0
2,11061,0
3,5032,0
4,17599,0


In [ ]:
lrpdf = pd.DataFrame(lrp)
sub_df = sam['enrollee_id']
sub_df = pd.concat([sub_df,lrpdf],axis =1)
sub_df.columns = ['enrollee_id','target']

In [ ]:
knpdf = pd.DataFrame(knnp)
sub_df = sam['enrollee_id']
sub_df = pd.concat([sub_df,knpdf],axis =1)
sub_df.columns = ['enrollee_id','target']

In [ ]:
rfpdf = pd.DataFrame(rfp)
sub_df = sam['enrollee_id']
sub_df = pd.concat([sub_df,rfpdf],axis =1)
sub_df.columns = ['enrollee_id','target']

In [48]:
xgpdf = pd.DataFrame(xgp)
sub_df = sam['enrollee_id']
sub_df = pd.concat([sub_df,xgpdf],axis =1)
sub_df.columns = ['enrollee_id','target']

In [46]:
catbdf = pd.DataFrame(catp)
sub_df = sam['enrollee_id']
sub_df = pd.concat([sub_df,catbdf],axis =1)
sub_df.columns = ['enrollee_id','target']

In [ ]:
sub_df.to_csv(r'E:\Pattabhiraman\Personal\JHHRA\lr03.csv', index=False)

In [ ]:
sub_df.to_csv(r'E:\Pattabhiraman\Personal\JHHRA\knn01.csv', index=False)

In [ ]:
sub_df.to_csv(r'E:\Pattabhiraman\Personal\JHHRA\rf02.csv', index=False)

In [49]:
sub_df.to_csv(r'E:\Pattabhiraman\Personal\JHHRA\xgb02.csv', index=False)

In [47]:
sub_df.to_csv(r'E:\Pattabhiraman\Personal\JHHRA\catb02.csv', index=False)